# R&D Cleanroom data

Clean R&D Cleanroom Data

data source: "rdc_data_raw.xlsx"

## 读取

In [1]:
import pandas as pd

In [2]:
!python -V

Python 3.10.13


In [4]:
df = pd.read_excel("./CCP/data/rdc_data_raw.xlsx")
df.head()

,ID,测风量,新风机设定频率（Hz）,送风机设定频率（Hz）,排风机设定频率（Hz）,一更送风阀（开度）,二更送风阀（开度）,洁净走廊1送风阀（开度）,洁净走廊2送风阀（开度）,测试间一送风阀（开度）,...,测试间一HEPA压差,测试间二HEPA压差,测试间三HEPA压差,一更HEPA风量,二更HEPA风量,洁净走廊1HEPA风量,洁净走廊2HEPA风量,测试间一HEPA风量,测试间二HEPA风量,测试间三HEPA风量
0,1,NaN,26,30,21,0,30,30,5,20,...,91.78,202.28,184.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,26,30,21,5,5,65,15,40,...,141.69,177.12,172.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,26,30,21,5,30,45,35,70,...,179.99,155.82,82.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,26,30,21,5,90,65,50,25,...,99.94,139.50,176.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1.0,26,30,21,10,75,70,85,60,...,161.07,95.17,165.14,186.0,324.0,323.0,321.0,654.0,382.0,675.0


## 清理数据

In [40]:
"""
Cell generated by Data Wrangler.
"""


def clean_data(df):
    # Replace missing values with 0 in column: '测风量'
    df = df.fillna({"测风量": 0})
    # 将“送风机设定频率（Hz）”列中的“待测”替换为同行的“B”列的值
    df["送风机设定频率（Hz）"] = df.apply(
        lambda x: x["送风机实际频率（Hz）"] if x["送风机设定频率（Hz）"] == "待测" else x["送风机设定频率（Hz）"],
        axis=1,
    )
    # 修正数据类型
    df["送风机设定频率（Hz）"] = df["送风机设定频率（Hz）"].astype("float64")
    # 将“系统送风量”列中大于10000的数据除以100
    df["系统送风量"] = df["系统送风量"].apply(lambda x: x / 100 if x > 10000 else x)
    # 将“系统新风量”列或“系统送风量”列中值为0的行删除
    df = df[(df["系统新风量"] != 0) | (df["系统送风量"] != 0)]
    # 将“系统回风量”列的值替换为“系统送风量”列值减去“系统新风量”列值
    df["系统回风量"] = df["系统送风量"] - df["系统新风量"]
    # 将“二更HEPA压差”列中大于10000的数据除以100
    df["二更HEPA压差"] = df["二更HEPA压差"].apply(lambda x: x / 100 if x > 10000 else x)
    # 删除无用特征
    df.drop(
        columns=[
            "ID",
            "测风量",
            "新风机设定频率（Hz）",
            "送风机设定频率（Hz）",
            "排风机设定频率（Hz）",
            "一更HEPA压差",
            "二更HEPA压差",
            "洁净走廊1HEPA压差",
            "洁净走廊2HEPA压差",
            "测试间一HEPA压差",
            "测试间二HEPA压差",
            "测试间三HEPA压差",
            "一更HEPA风量",
            "二更HEPA风量",
            "洁净走廊1HEPA风量",
            "洁净走廊2HEPA风量",
            "测试间一HEPA风量",
            "测试间二HEPA风量",
            "测试间三HEPA风量",
        ],
        inplace=True,
    )
    return df


df_cleaned = clean_data(df.copy())
df_cleaned.head()

,一更送风阀（开度）,二更送风阀（开度）,洁净走廊1送风阀（开度）,洁净走廊2送风阀（开度）,测试间一送风阀（开度）,测试间二送风阀（开度）,测试间三送风阀（开度）,二更回风阀（开度）,洁净走廊回风阀（开度）,测试间一回风阀（开度）,...,系统新风量,系统送风量,系统回风量,系统排风量,一更压差,二更压差,洁净走廊压差,测试间一压差,测试间二压差,测试间三压差
0,0,30,30,5,20,80,60,90,70,5,...,1751.39,2814.99,1063.60,1370.09,0.86,18.00,17.58,22.21,26.72,33.74
1,5,5,65,15,40,65,60,90,70,5,...,1851.88,2924.85,1072.97,1454.92,3.11,8.17,16.00,27.43,29.04,28.67
2,5,30,45,35,70,45,20,90,70,5,...,1849.72,2898.06,1048.34,1248.05,5.31,19.06,18.14,30.42,28.60,3.63
3,5,90,65,50,25,40,90,90,70,5,...,1877.58,2971.16,1093.58,1324.64,4.69,20.30,20.96,18.32,19.39,30.31
4,10,75,70,85,60,25,85,90,70,5,...,1838.90,3096.50,1257.60,1302.34,6.79,18.18,18.73,19.46,16.62,25.89


In [41]:
df_cleaned.columns

Index(['一更送风阀（开度）', '二更送风阀（开度）', '洁净走廊1送风阀（开度）', '洁净走廊2送风阀（开度）', '测试间一送风阀（开度）',
       '测试间二送风阀（开度）', '测试间三送风阀（开度）', '二更回风阀（开度）', '洁净走廊回风阀（开度）', '测试间一回风阀（开度）',
       '测试间二回风阀（开度）', '测试间三固排风阀（开度）', '测试间一排风阀（开度）', '测试间二排风阀（开度）',
       '测试间三排风阀（开度）', '新风机实际频率（Hz）', '送风机实际频率（Hz）', '排风机实际频率（Hz）', '系统新风量',
       '系统送风量', '系统回风量', '系统排风量', '一更压差', '二更压差', '洁净走廊压差', '测试间一压差', '测试间二压差',
       '测试间三压差'],
      dtype='object')

## 列名统一

In [42]:
# 将 df 的列名按照字典替换
rplc_dict = {
    "ID": "ID",
    "一更送风阀（开度）": "RM1_SUPP_DMPR_0",
    "二更送风阀（开度）": "RM2_SUPP_DMPR_0",
    "洁净走廊1送风阀（开度）": "RM6_SUPP_DMPR_0",
    "洁净走廊2送风阀（开度）": "RM6_SUPP_DMPR_1",
    "测试间一送风阀（开度）": "RM3_SUPP_DMPR_0",
    "测试间二送风阀（开度）": "RM4_SUPP_DMPR_0",
    "测试间三送风阀（开度）": "RM5_SUPP_DMPR_0",
    "二更回风阀（开度）": "RM2_RET_DMPR_0",
    "洁净走廊回风阀（开度）": "RM6_RET_DMPR_0",
    "测试间一回风阀（开度）": "RM3_RET_DMPR_0",
    "测试间二回风阀（开度）": "RM4_RET_DMPR_0",
    "测试间三固排风阀（开度）": "RM5_EXH_DMPR_1",
    "测试间一排风阀（开度）": "RM3_EXH_DMPR_0",
    "测试间二排风阀（开度）": "RM4_EXH_DMPR_0",
    "测试间三排风阀（开度）": "RM5_EXH_DMPR_0",
    "新风机实际频率（Hz）": "MAU_FREQ",
    "送风机实际频率（Hz）": "AHU_FREQ",
    "排风机实际频率（Hz）": "EF_FREQ",
    "系统新风量": "TOT_FRSH_VOL",
    "系统送风量": "TOT_SUPP_VOL",
    "系统回风量": "TOT_RET_VOL",
    "系统排风量": "TOT_EXH_VOL",
    "一更压差": "RM1_PRES",
    "二更压差": "RM2_PRES",
    "洁净走廊压差": "RM6_PRES",
    "测试间一压差": "RM3_PRES",
    "测试间二压差": "RM4_PRES",
    "测试间三压差": "RM5_PRES",
}
df_cleaned.rename(columns=rplc_dict, inplace=True)
df_cleaned.head()

,RM1_SUPP_DMPR_0,RM2_SUPP_DMPR_0,RM6_SUPP_DMPR_0,RM6_SUPP_DMPR_1,RM3_SUPP_DMPR_0,RM4_SUPP_DMPR_0,RM5_SUPP_DMPR_0,RM2_RET_DMPR_0,RM6_RET_DMPR_0,RM3_RET_DMPR_0,...,TOT_FRSH_VOL,TOT_SUPP_VOL,TOT_RET_VOL,TOT_EXH_VOL,RM1_PRES,RM2_PRES,RM6_PRES,RM3_PRES,RM4_PRES,RM5_PRES
0,0,30,30,5,20,80,60,90,70,5,...,1751.39,2814.99,1063.60,1370.09,0.86,18.00,17.58,22.21,26.72,33.74
1,5,5,65,15,40,65,60,90,70,5,...,1851.88,2924.85,1072.97,1454.92,3.11,8.17,16.00,27.43,29.04,28.67
2,5,30,45,35,70,45,20,90,70,5,...,1849.72,2898.06,1048.34,1248.05,5.31,19.06,18.14,30.42,28.60,3.63
3,5,90,65,50,25,40,90,90,70,5,...,1877.58,2971.16,1093.58,1324.64,4.69,20.30,20.96,18.32,19.39,30.31
4,10,75,70,85,60,25,85,90,70,5,...,1838.90,3096.50,1257.60,1302.34,6.79,18.18,18.73,19.46,16.62,25.89


## 导出数据

In [43]:
# 导出为 csv
df_cleaned.to_csv("data/rdc_data_cleaned.csv")